# Úkol č. 1 - vizualizace dat a web scraping

  * V rámci tohoto úkolu musíte opanovat nástroje pro stahování dat z webu (tzv. _web scraping_) a následně data zpracovat a vizualizovat. 
  * Vaším úkolem bude stáhnout a zpracovat data týkající se kandidátů v minulých volbách do Evropského parlamentu.
  
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Zdroje dat

  * Primárním zdrojem dat je tato tabulka: https://volby.cz/pls/ep2014/ep2111?xjazyk=CZ&xv=1&xt=2&xstrana=0

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **10 bodů**:
  * Stáhněte data z výše uvedeného a uložte je do tabulky ve formátu _csv_.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Věkovou skladbu kandidátů a kandidátek jednotlivých stran.
    * Vzdělanostní skladbu kadidátů a kadidátek jednotlivých stran: pokuste se z titulů odhadnout 
    * Žebříčky nejůspěšňějších kandidátů a kandidátek (zaměřte se na preferenční hlasy a nějak zvažte jejich váhu vzhledem k počtu hlasů strany samotné).
    * Pokuste se odhadnout počty žen kandidujících za jednotlivé strany.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 15 bodů):
  * (až +4 body) Najděte nějaký nástroj, který Vám umožní do notebooku přidat nějaké interaktivní prvky (např. si uživatel vybere ze seznamu stranu a notebook automaticky zobrazí vizualizace pouze vybraných kandidátů)
  * (až +4 body) Zpracujte i data z předešlých voleb do EP z roku 2009 a identifikujte kandidáty a kandidátky, kteří kandidovaly v obou volbách. I toto zkuste nějak rozumně vizualizovat.
  
## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/MI-PDM/homeworks/index.html.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

## 01 Načtení dat

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup

url = "https://volby.cz/pls/ep2014/ep2111?xjazyk=CZ&xv=1&xt=2&xstrana=0"
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table")
rows = []

headers = [
    "CandidateListID",
    "CandidateListName",
    "CandidateOrder",
    "CandidateName",
    "CandidateAge",
    "FilingParty",
    "PoliticalAffiliation",
    "PreferentialVotesAbs",
    "PreferentialVotesPercentage",
    "Mandate",
    "Order"
]

for row in table.findAll("tr"):
    cells = []
    
    for cell in row.findAll("td"):
        text = cell.text.replace(",", ".")
        cells.append(text)
    
    rows.append(cells)

with open("volby.csv", "w", encoding="utf-8") as output:
    output.write(",".join(headers))
    output.write("\n")
    for row in rows:
        if not row:
            continue
        output.write(",".join(row))
        output.write("\n")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline 
matplotlib.style.use('ggplot')

In [ ]:
import pandas as pd

data = pd.read_csv("volby.csv", encoding="utf-8")

## 02 Věková skladba kandidátů

In [ ]:
data["CandidateAge"].hist();

In [ ]:
number_of_lists = data["CandidateListID"].max()
height = 6 * number_of_lists
fig, ax = plt.subplots(number_of_lists, 1, figsize=(10, height))

for list in data["CandidateListID"].unique():
    index = list - 1
    list_data = data[data["CandidateListID"] == list]
    list_data["CandidateAge"].hist(ax=ax[index])
    ax[index].set_title(list_data["CandidateListName"].iloc[0])
    ax[index].set_xlabel("Věk kandidátů")
    ax[index].set_ylabel("Počet kandidátů")

## 03 Vzdělání kandidátů

In [ ]:
dis = ["DiS"]
bc = ["Bc", "BcA", "BBA", "BPA", "B.Th.", "B.A.", "BSc.", "LL.B.", "BLaw"]
mgr = ["Ing.", "Ing. arch.", "MUDr.", "MDDr.", "MVDr.", "MgA.", "Mgr.", "JUDr.", "PhDr.", "RNDr.", "PharmDr.", "ThLic.", "ThDr.", "MSDr.", "PaedDr.", "PhMr.", "RCDr.", "RSDr.", "RTDr.", "ThMgr.", "MBA", "MPA", "M.Th.", "M.A.", "MSc.", "Dipl.-Ing.", "MLaw", "LL.M.", "MMed."]
doc = ["Ph.D.", "DSc.", "CSc.", "Dr.", "DrSc.", "Th.D.", "M.D.", "EngD."]

data["Education"] = 0

for index, row in data.iterrows():
    name = row["CandidateName"]
    if sum([x in name for x in doc]) > 0:
        data.at[index, "Education"] = 4
    elif sum([x in name for x in mgr]) > 0:
        data.at[index, "Education"] = 3
    elif sum([x in name for x in bc]) > 0:
        data.at[index, "Education"] = 2
    elif sum([x in name for x in dis]) > 0:
        data.at[index, "Education"] = 1
    else:
        continue

In [ ]:
max_value = data["Education"].max()
min_value = data["Education"].min()
bins = max_value - min_value + 1
bin_length = (bins - 1) / bins
ticks = [min_value + bin_length * (x - 1 / 2) for x in range(1, bins + 1)]
plt.hist(data["Education"], bins=bins)
plt.xticks(ticks, ["Maturita", "DiS", "Bc", "Mgr", "PhD"]);

In [ ]:
number_of_lists = data["CandidateListID"].max()
height = 6 * number_of_lists
fig, ax = plt.subplots(number_of_lists, 1, figsize=(10, height))
max_value = data["Education"].max()
min_value = data["Education"].min()
bins = max_value - min_value + 1
bin_length = (bins - 1) / bins
ticks = [min_value + bin_length * (x - 1 / 2) for x in range(1, bins + 1)]
plt.setp(ax, xticks=ticks, xticklabels=["Maturita", "DiS", "Bc", "Mgr", "PhD"])

for list in data["CandidateListID"].unique():
    index = list - 1
    list_data = data[data["CandidateListID"] == list]
    list_data["Education"].hist(ax=ax[index], bins=bins, range=[min_value, max_value])
    ax[index].set_title(list_data["CandidateListName"].iloc[0])
    ax[index].set_xlabel("Vzdělání kandidátů")
    ax[index].set_ylabel("Počet kandidátů")

## 04 Úspěšnost kandidátů

In [ ]:
# Pokud seřadím kandidáty podle procentuálního počtu preferenčních hlasů,
# získám tak nejúspěšnější kandidáty v rámci strany podle preferenčních hlasů
data.sort_values("PreferentialVotesPercentage", ascending=False).head(10)

## 05 Počet kandidujících žen

In [ ]:
women = data[data["CandidateName"].str.contains("ová ")]
max_value = data["CandidateListID"].max()
min_value = data["CandidateListID"].min()
ticks = [x + 1 / 2 for x in range(min_value, max_value + 1)]
indexes = data["CandidateListID"].unique()
names = [data[data["CandidateListID"] == x]["CandidateListName"].iloc[0] for x in indexes]
plt.figure(figsize=(10, 20))
plt.hist(women["CandidateListID"], orientation="horizontal", bins=len(names))
plt.yticks(ticks, names);